<a href="https://colab.research.google.com/github/lis-r-barreto/Data-Engineering-Portfolio/blob/main/02_Exploring_Hacker_News_Posts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Explorando Postagens do Hacker News**

*(Exploring Hacker News Posts)*


<p align="center">
  <img src="https://miro.medium.com/max/700/1*CVOGx9ckrpWyTvtQgauYpw.jpeg" width="300" >
</p>

## **Introdução**

O Hacker News é um site iniciado pela incubadora de startups [Y Combinator](https://www.ycombinator.com/), onde as histórias enviadas por usuários (conhecidas como "posts") são votadas e comentadas, semelhante ao reddit. O Hacker News é extremamente popular nos círculos de tecnologia e startups, e as postagens que chegam ao topo das listagens do Hacker News podem receber centenas de milhares de visitantes como resultado.


## **Objetivo**
Estamos especificamente interessados ​​em postagens cujos títulos começam com Ask HN ou Show HN. Os usuários enviam postagens Ask HN para fazer uma pergunta específica à comunidade Hacker News. Abaixo segue um exemplo:

*Ask HN: Como melhorar meu site pessoal?*

Da mesma forma, os usuários enviam postagens do Show HN para mostrar à comunidade do Hacker News um projeto, produto ou apenas algo interessante. Abaixo segue um exemplo:

*Show HN: Plataforma de desenvolvimento de hardware baseada em Wio Link ESP8266 Web of Things'*

Compararemos esses dois tipos de postagens para determinar o seguinte:
- Em média, quais postagens recebem mais comentários? Os posts cujos títulos começam com Ask HN ou Show HN?
- Em média, qual horário recebe uma quantidade maior de comentários?

## **Abrindo e explorando os dados**

Você pode encontrar o conjunto de dados [aqui](https://www.kaggle.com/hacker-news/hacker-news-posts), mas observe que ele foi reduzido de quase 300.000 linhas para aproximadamente 20.000 linhas removendo todos os envios que não recebeu nenhum comentário e, em seguida, amostragem aleatória das submissões restantes.

Abaixo estão as descrições das colunas:
- id: O identificador exclusivo do Hacker News para a postagem
- título: O título do post
- url: o URL para o qual as postagens são vinculadas, se a postagem tiver uma URL
- num_points: o número de pontos que o post adquiriu, calculado como o número total de votos positivos menos o número total de votos negativos
- num_comments: O número de comentários que foram feitos no post
- autor: o nome de usuário da pessoa que enviou a postagem
- created_at: A data e hora em que a postagem foi enviada


Vamos começar importando as bibliotecas que precisamos e lendo o conjunto de dados em uma lista de listas e explorando as primeiras 5 linhas.

In [2]:
# Download do http para arquivo local
!wget --quiet --show-progress https://raw.githubusercontent.com/saraheldawody/Exploring-Hacker-News-Posts/master/hacker_news.csv

hacker_news.csv.1   100%[===================>]   2.96M  --.-KB/s    in 0.07s   


In [3]:
from csv import reader

opened_file = open ('hacker_news.csv',encoding = 'utf8')
read_file = reader (opened_file)
hn = list (read_file)

for row in hn[:5]:
    print (row)
    print('\n')

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']


['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']


['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']


['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']




Observe que a primeira lista nas listas internas contém os cabeçalhos das colunas e as listas posteriores contêm os dados de uma linha. Para analisar nossos dados, precisamos primeiro remover a linha que contém os cabeçalhos das colunas. Vamos remover essa primeira linha.

In [4]:
headers = hn[0]  #Não execute esta célula mais de uma vez
hn = hn[1:]

print (headers)
print('\n')
for row in hn[:5]:
    print (row)
    print('\n')

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']


['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']


['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']


['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']


['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']




Agora que removemos os cabeçalhos de hn, estamos prontos para filtrar nossos dados. Como estamos preocupados apenas com títulos de postagem que começam com **Ask HN** ou **Show HN**, criaremos novas listas de listas contendo apenas os dados desses títulos.

## **Classificando os dados**

Para encontrar as postagens que começam com Ask HN ou Show HN (e variações de maiúsculas e minúsculas), usaremos o método string `startswith` e verificaremos os dados imprimindo cinco linhas de cada lista.

In [5]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    title_lower = title.lower()
    
    if title_lower.startswith('ask hn'):
        ask_posts.append(row)
    
    elif title_lower.startswith('show hn'):
        show_posts.append(row)
        
    else:
        other_posts.append(row)
        
for row in ask_posts[:5]:
    print (row)
    print('\n')
print('\n')
for row in show_posts[:5]:
    print (row)
    print('\n')
print('\n')
for row in other_posts[:5]:
    print (row)
    print('\n')
print('Número de posts Ask HN:',len(ask_posts))
print('Número de posts Show HN:',len(show_posts))
print('Número de outros posts:',len(other_posts))

['12296411', 'Ask HN: How to improve my personal website?', '', '2', '6', 'ahmedbaracat', '8/16/2016 9:55']


['10610020', 'Ask HN: Am I the only one outraged by Twitter shutting down share counts?', '', '28', '29', 'tkfx', '11/22/2015 13:43']


['11610310', 'Ask HN: Aby recent changes to CSS that broke mobile?', '', '1', '1', 'polskibus', '5/2/2016 10:14']


['12210105', 'Ask HN: Looking for Employee #3 How do I do it?', '', '1', '3', 'sph130', '8/2/2016 14:20']


['10394168', 'Ask HN: Someone offered to buy my browser extension from me. What now?', '', '28', '17', 'roykolak', '10/15/2015 16:38']




['10627194', 'Show HN: Wio Link  ESP8266 Based Web of Things Hardware Development Platform', 'https://iot.seeed.cc', '26', '22', 'kfihihc', '11/25/2015 14:03']


['10646440', 'Show HN: Something pointless I made', 'http://dn.ht/picklecat/', '747', '102', 'dhotson', '11/29/2015 22:46']


['11590768', 'Show HN: Shanhu.io, a programming playground powered by e8vm', 'https://shanhu.io', '1', 

Em seguida, vamos determinar se as postagens de perguntas ou de exibição recebem mais comentários em média.

In [6]:
total_ask_comments = 0

for row in ask_posts:
    total_ask_comments +=  int(row[4])
    
avg_ask_comments = total_ask_comments / len (ask_posts)

total_show_comments = 0

for row in show_posts:
    total_show_comments +=  int(row[4])
    
avg_show_comments = total_show_comments / len (show_posts)

print('Número médio de comentários Ask HN:',avg_ask_comments)
print('Número médio de comentários Show HN:',avg_show_comments)

Número médio de comentários Ask HN: 14.038417431192661
Número médio de comentários Show HN: 10.31669535283993


## **Reformatando e analisando os dados**

Fica claro que, em média, os posts de pergunta recebem mais comentários do que os posts de exibição. Como as postagens de perguntas são mais propensas a receber comentários, focaremos nossa análise restante apenas nessas postagens.

Em seguida, determinaremos se as postagens de perguntas criadas em um determinado momento são mais propensas a atrair comentários. Usaremos as seguintes etapas para realizar essa análise:

1. Calcular a quantidade de posts Ask HN criados em cada hora do dia, juntamente com o número de comentários recebidos.
2. Calcular o número médio de comentários que as postagens de solicitação recebem por hora de criação.

Usaremos o módulo `datetime` para trabalhar com os dados na coluna ` created_at` para calcular a quantidade de postagens e comentários de perguntas por hora criados.

In [7]:
import datetime as dt

result_list = []

for row in ask_posts:
    result_list.append ( [ row[6] , int(row[4]) ] )
    
counts_by_hour = {}
comments_by_hour = {}

for row in result_list:
    time_created = dt.datetime.strptime(row[0],'%m/%d/%Y %H:%M')
    hour = time_created.hour
    
    if hour not in counts_by_hour:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = row[1]
    else:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += row[1]

print('O número de postagens de perguntas criadas durante cada hora do dia:')
print(counts_by_hour)
print('\n')
print('O número correspondente de comentários criados a cada hora recebida:')
print(comments_by_hour)

O número de postagens de perguntas criadas durante cada hora do dia:
{9: 45, 13: 85, 10: 59, 14: 107, 16: 108, 23: 68, 12: 73, 17: 100, 15: 116, 21: 109, 20: 80, 2: 58, 18: 109, 3: 54, 5: 46, 19: 110, 1: 60, 22: 71, 8: 48, 4: 47, 0: 55, 6: 44, 7: 34, 11: 58}


O número correspondente de comentários criados a cada hora recebida:
{9: 251, 13: 1253, 10: 793, 14: 1416, 16: 1814, 23: 543, 12: 687, 17: 1146, 15: 4477, 21: 1745, 20: 1722, 2: 1381, 18: 1439, 3: 421, 5: 464, 19: 1188, 1: 683, 22: 479, 8: 492, 4: 337, 0: 447, 6: 397, 7: 267, 11: 641}


Em seguida, usaremos esses dois dicionários para calcular o número médio de comentários para postagens criadas durante cada hora do dia, criando uma lista de listas contendo as horas em que as postagens foram criadas e o número médio de comentários que essas postagens receberam.

In [8]:
avg_by_hour = []

for hour in counts_by_hour:
    no_of_posts = counts_by_hour[hour]
    no_of_comments = comments_by_hour[hour]
    avg_no_comments = comments_by_hour[hour] / counts_by_hour[hour] 
    avg_by_hour.append([hour,avg_no_comments])
    
print(avg_by_hour)

[[9, 5.5777777777777775], [13, 14.741176470588234], [10, 13.440677966101696], [14, 13.233644859813085], [16, 16.796296296296298], [23, 7.985294117647059], [12, 9.41095890410959], [17, 11.46], [15, 38.5948275862069], [21, 16.009174311926607], [20, 21.525], [2, 23.810344827586206], [18, 13.20183486238532], [3, 7.796296296296297], [5, 10.08695652173913], [19, 10.8], [1, 11.383333333333333], [22, 6.746478873239437], [8, 10.25], [4, 7.170212765957447], [0, 8.127272727272727], [6, 9.022727272727273], [7, 7.852941176470588], [11, 11.051724137931034]]


Agora vamos classificar os dados acima em ordem decrescente do número médio de comentários para facilitar a análise.

In [9]:
swap_avg_by_hour = []

for value in avg_by_hour:
    swap_avg_by_hour.append([value[1],value[0]])
    
sorted_swap = sorted(swap_avg_by_hour,reverse = True)

print('Os 5 horários com maior numéro médio de comentários por postagens do Ask HN')
print('\n')

for row in sorted_swap:
    template = '{time}:00: {num:.2f} média de comentários por post.'
    print(template.format(time = row[1] , num = row[0]))

Os 5 horários com maior numéro médio de comentários por postagens do Ask HN


15:00: 38.59 média de comentários por post.
2:00: 23.81 média de comentários por post.
20:00: 21.52 média de comentários por post.
16:00: 16.80 média de comentários por post.
21:00: 16.01 média de comentários por post.
13:00: 14.74 média de comentários por post.
10:00: 13.44 média de comentários por post.
14:00: 13.23 média de comentários por post.
18:00: 13.20 média de comentários por post.
17:00: 11.46 média de comentários por post.
1:00: 11.38 média de comentários por post.
11:00: 11.05 média de comentários por post.
19:00: 10.80 média de comentários por post.
8:00: 10.25 média de comentários por post.
5:00: 10.09 média de comentários por post.
12:00: 9.41 média de comentários por post.
6:00: 9.02 média de comentários por post.
0:00: 8.13 média de comentários por post.
23:00: 7.99 média de comentários por post.
7:00: 7.85 média de comentários por post.
3:00: 7.80 média de comentários por post.
4:00: 7.17 m

## **Conclusão**

- As postagens de perguntas têm um número médio de comentários mais alto do que as postagens de exibição.
- 15:00 GMT tem o maior número médio de comentários.


### **Este projeto guiado faz parte do [*Data Enginnering Carreer Path*](https://www.dataquest.io/path/data-engineer/) proposto pela**

<p align="center">
  <img src="https://www.bootcamprating.com/wp-content/uploads/2020/09/Dataquest_school_logo_image.png" width="200" >
</p>